In [1]:
import os
import json
import warnings 
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


# Ignore all warnings
warnings.filterwarnings("ignore")

# ------------- FEATURE ENGINEERING -------------
def make_features(df):
    s  = df["Strain"].astype(float).values
    sr = df["Strain rate (s-1)"].astype(float).values
    T  = df["Deformation Temperature"].astype(float).values

    X = pd.DataFrame({
        "s": s,
        "sr": sr,
        "T": T,
        "s2": s**2,
        "s3": s**3,
        "sr2": sr**2,
        "sr3": sr**3,
        "s_sr": s * sr,
        "s_T": s * T,
        "sr_T": sr * T,
        "log_s": np.log1p(np.abs(s)),
        "log_sr": np.log1p(sr),
        "invT": 1/(T + 273.15),
        "exp_invT": np.exp(1/(T + 273.15)),
        "s_over_sr": s/(sr+1e-8),
        "sr_over_s": sr/(s+1e-8)
    })
    return X.values


# ------------- MATRIX EXPORT ---------------------------------------------------------->
def export_matrix(model, x_scaler, y_scaler, out_path="ANN_Matrix.json"):
    matrix = {
        "x_min": x_scaler.data_min_.tolist(),
        "x_max": x_scaler.data_max_.tolist(),
        "y_min": float(y_scaler.data_min_[0]),
        "y_max": float(y_scaler.data_max_[0]),
        "layers": []
    }

    for layer in model.layers:
        weights = layer.get_weights()
        if len(weights) == 2:
            W, b = weights
            matrix["layers"].append({
                "weights": W.tolist(),
                "bias": b.tolist(),
                "activation": layer.activation.__name__   # "tanh" or "linear"
            })

    with open(out_path, "w") as f:
        json.dump(matrix, f, indent=4)

    print(f"\nMatrix saved → {out_path}")


# ------------- MAIN TRAINING -------------
def main():
    df = pd.read_csv("Training_data.csv").dropna()

    X_raw = make_features(df)
    y_raw = df["FlowStress"].astype(float).values.reshape(-1, 1)

    # Scaling
    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    X = x_scaler.fit_transform(X_raw)
    y = y_scaler.fit_transform(y_raw)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    neuron_options = [8, 12, 16, 20, 24]
    best_val_loss = float("inf")
    best_model = None
    best_n = None

    print("\n========== NEURON SEARCH START ==========")

    for n in neuron_options:
        print(f"\nTraining model with {n} neurons...")

        model = Sequential([
            Dense(n, activation="tanh", input_shape=(X.shape[1],)),
            Dense(n * 2, activation="tanh"),
            Dense(n, activation="tanh"),
            Dense(1, activation="linear")
        ])

        model.compile(optimizer="adam", loss="mse")

        es = EarlyStopping(
            monitor="val_loss",
            patience=50,
            restore_best_weights=True
        )

        history = model.fit(
            X_train, y_train,
            validation_split=0.2,
            epochs=700,
            batch_size=32,
            verbose=0,
            callbacks=[es]
        )

        val_loss = min(history.history["val_loss"])
        print(f"Best val_loss for {n} neurons = {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model
            best_n = n

    print("\n========== SEARCH COMPLETE ==========")
    print(f"Best neurons = {best_n}, best val_loss = {best_val_loss:.6f}")

    # Test performance on hold-out set
    y_test_pred = best_model.predict(X_test).ravel()
    y_test_true = y_test.ravel()

    r2 = r2_score(y_test_true, y_test_pred)
    mse = mean_squared_error(y_test_true, y_test_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_true, y_test_pred)

    print("\nPerformance on scaled test set:")
    print(f"R2   = {r2:.6f}")
    print(f"MAE  = {mae:.6f}")
    print(f"RMSE = {rmse:.6f}")

    # Save model and matrix
    best_model.save("ANN_Model.keras")
    print("\nBest Keras model saved → ANN_Model.h5")

    export_matrix(best_model, x_scaler, y_scaler, "ANN_Matrix.json")
    print("\nProgram-1 finished successfully.\n")


if __name__ == "__main__":
    main()



========== NEURON SEARCH START ==========

Training model with 8 neurons...
Best val_loss for 8 neurons = 0.002201

Training model with 12 neurons...
Best val_loss for 12 neurons = 0.001252

Training model with 16 neurons...
Best val_loss for 16 neurons = 0.001262

Training model with 20 neurons...
Best val_loss for 20 neurons = 0.001422

Training model with 24 neurons...
Best val_loss for 24 neurons = 0.001439

========== SEARCH COMPLETE ==========
Best neurons = 12, best val_loss = 0.001252
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step

Performance on scaled test set:
R2   = 0.899390
MAE  = 0.032943
RMSE = 0.040212

Best Keras model saved → ANN_Model.h5

Matrix saved → ANN_Matrix.json

Program-1 finished successfully.

